In [38]:
from abc import ABC, abstractmethod
from dataclasses import dataclass

In [41]:
class Expression(ABC):

    def __invert__(self) -> 'Not':
        return Not(self)

    def __and__(self, other) -> 'And':
        return And(self, other)

    def __or__(self, other) -> 'Or':
        return Or(self, other)

    @abstractmethod
    def calculate(self) -> 'Expression':
        ...


@dataclass(frozen=True)
class Literal(Expression):
    name: str

    def __str__(self):
        return self.name

    def __repr__(self):
        return f'Literal({self.name})'

    def calculate(self):
        return self


_0 = Literal('0')
_1 = Literal('1')


@dataclass(frozen=True)
class Not(Expression):
    expr: Expression

    def __str__(self):
        template = '~(%s)' \
            if isinstance(self.expr, And | Or) \
            else '~%s'
        return template % self.expr

    def __repr__(self):
        return f'Not({repr(self.expr)})'

    def calculate(self):
        result = self.expr.calculate()
        if result == _0:
            return _1
        if result == _1:
            return _0
        return Not(result)


@dataclass(frozen=True)
class And(Expression):
    left_expr: Expression
    right_expr: Expression

    def __str__(self):
        left_template = '(%s)' \
            if isinstance(self.left_expr, Or) \
            else '%s'
        right_template = '(%s)' \
            if isinstance(self.right_expr, Or) \
            else '%s'
        template = left_template + ' & ' + right_template
        return template % (self.left_expr, self.right_expr)

    def __repr__(self):
        return f'And({repr(self.left_expr)}, {repr(self.right_expr)})'

    def calculate(self):
        left_result = self.left_expr.calculate()
        right_result = self.right_expr.calculate()
        if _0 in (left_result, right_result):
            return _0
        if left_result == _1:
            return right_result
        if right_result == _1:
            return left_result
        return And(left_result, right_result)


@dataclass(frozen=True)
class Or(Expression):
    left_expr: Expression
    right_expr: Expression

    def __str__(self):
        left_template = '(%s)' \
            if isinstance(self.left_expr, Or) \
            else '%s'
        right_template = '(%s)' \
            if isinstance(self.right_expr, Or) \
            else '%s'
        template = left_template + ' | ' + right_template
        return template % (self.left_expr, self.right_expr)

    def __repr__(self):
        return f'Or({repr(self.left_expr)}, {repr(self.right_expr)})'

    def calculate(self):
        left_result = self.left_expr.calculate()
        right_result = self.right_expr.calculate()
        if _1 in (left_result, right_result):
            return _1
        if left_result == _0:
            return right_result
        if right_result == _0:
            return left_result
        return Or(left_result, right_result)



In [42]:
for lit in 'ABCD':
    exec(f'{lit} = Literal("{lit}")')

In [45]:
e1 = A&B|C&D&_0
print(e1)
e1

A & B | C & D & 0


Or(And(Literal(A), Literal(B)), And(And(Literal(C), Literal(D)), Literal(0)))

In [47]:
e2 = e1.calculate()
print(e2)
e2

A & B


And(Literal(A), Literal(B))